<a href="https://colab.research.google.com/github/lukaszplust/Projects/blob/main/Btree_DB_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
class Record:
    def __init__(self, key, data):
        self.key = key
        self.data = data

    def __repr__(self):
        return f"Record({self.key}, {self.data})"


    # to_bytes - metoda konwertująca obiekt Record na bajty, co jest przydatne do zapisywania danych w pliku

    def to_bytes(self):
        # konwertuje klucz key na 4-bajtowy zapis w porządku big-endian
        key_bytes = self.key.to_bytes(4, byteorder='big')
        # koduje dane data na bajty (domyślnie przy użyciu kodowania UTF-8)
        data_bytes = self.data.encode()
        # obliczam długość data_bytes i konwertuje ją na 4-bajtowy zapis w porządku big-endian
        length_bytes = len(data_bytes).to_bytes(4, byteorder='big')
        # zwracam jedną sekwencję bajtów
        return key_bytes + length_bytes + data_bytes

    # dekorator oznaczający, że metoda nie zależy od instancji klasy i może być wywoływana na poziomie klasy
    @staticmethod
    def from_bytes(byte_data):
        # konwertuje pierwsze 4 bajty byte_data na liczbę całkowitą w porządku big-endian
        key = int.from_bytes(byte_data[:4], byteorder='big')
        # konwertuje kolejne 4 bajty byte_data na liczbę całkowitą, reprezentującą długość danych
        length = int.from_bytes(byte_data[4:8], byteorder='big')
        # dekoduje bajty reprezentujące dane z byte_data na string
        data = byte_data[8:8+length].decode()
        # tworze i zwracam nowy obiekt Record z kluczem i danymi odczytanymi z bajtów
        return Record(key, data)



class ReadBuffer:
    def __init__(self, path, buffer_size=4):
        # przechowuje ścieżkę do pliku
        self.path = path
        # ustalam rozmiar bufora, czyli liczbę rekordów, które będą trzymane w pamięci
        self.buffer_size = buffer_size
        # inicjalizuje pustą listę bufora
        self.buffer = []
        # otwieram plik w trybie binarnym do odczytu
        self.file = open(self.path, "rb")
        # ustalam początkową pozycję w buforze
        self.position = 0
        # inicjalizuje licznik operacji odczytu z dysku
        self.disk_reads_count = 0

    # metoda odczytująca następny rekord z bufora
    def read_next(self):
        # sprawdzam, czy aktualna pozycja jest poza zakresem bufora
        if self.position >= len(self.buffer):
            # jeśli bufor jest pusty lub trzeba go uzupełnić, wywołuje metodę _fill_buffer
            self._fill_buffer()
        # sprawdzam, czy bufor jest pusty po jego uzupełnieniu
        if not self.buffer:
            # jeśli bufor jest pusty, zwraca None
            return None
        # pobiera rekord z aktualnej pozycji w buforze
        record = self.buffer[self.position]
        # zwiększam pozycję w buforze
        self.position += 1
        # zwracam pobrany rekord
        return record

    # metoda (prywatna) uzupełniająca bufor danymi z pliku
    def _fill_buffer(self):
        # czyszcze bufor
        self.buffer = []
        # resetuje pozycję w buforze
        self.position = 0
        # iteruje przez rozmiar bufora
        for _ in range(self.buffer_size):
            # odczytuje 8 bajtów z pliku
            bytes_record = self.file.read(8)
            # sprawdzam, czy odczytano dane
            if not bytes_record:
                # jeśli nie, kończe pętlę
                break
            # odczytuje długość danych z bajtów
            length = int.from_bytes(bytes_record[4:8], byteorder='big')
            # odczytuje dane o długości length
            data = self.file.read(length)
            # tworze rekord z bajtów i dodaje go do bufora
            self.buffer.append(Record.from_bytes(bytes_record + data))
        # zwiększam licznik operacji odczytu
        self.disk_reads_count += 1

    # metoda zamykająca plik
    def close(self):
        self.file.close()


class WriteBuffer:
    def __init__(self, path, buffer_size=4):
        # przechowuje ścieżkę do pliku
        self.path = path
        # ustalam rozmiar bufora, czyli liczbę rekordów, które będą trzymane w pamięci
        self.buffer_size = buffer_size
        # inicjalizuje pustą listę bufora
        self.buffer = []
        # inicjalizuje licznik operacji zapisu na dysku
        self.disk_writes_count = 0
    # metoda zapisująca rekord do bufora
    def write(self, record):
        # dodaje rekord do bufora
        self.buffer.append(record)
        # sprawdzam, czy bufor osiągnął rozmiar buffer_size
        if len(self.buffer) >= self.buffer_size:
            # jeśli bufor jest pełny, zapisuje jego zawartość na dysku
            self.flush()

    # metoda zapisująca zawartość bufora do pliku
    def flush(self):
        # otwieram plik w trybie dodawania binarnego ("ab")
        with open(self.path, "ab") as file:
            # iteruje przez rekordy w buforze
            for record in self.buffer:
                # zapisuje każdy rekord jako bajty do pliku
                file.write(record.to_bytes())
        # zwiększam licznik operacji zapisu
        self.disk_writes_count += 1
        # czyszcze bufor po zapisaniu
        self.buffer = []

    # metoda zamykająca bufor, zapisując pozostałe rekordy
    def close(self):
        # sprawdzam, czy w buforze są jakieś dane
        if self.buffer:
            # jeśli są dane, to zapisuje je do pliku
            self.flush()


class BTreeNode:
    def __init__(self, t, leaf=False):
        # minimalny stopień drzewa (d tu dałem t), który determinuje
        #minimalną i maksymalną liczbę kluczy, które węzeł może zawierać
        self.t = t

        # flaga oznaczająca, czy węzeł jest liściem (brak dzieci)
        self.leaf = leaf
        # lista kluczy przechowywanych w węźle
        self.keys = []
        # lista dzieci (węzłów) tego węzła, jeśli węzeł nie jest liściem
        self.children = []

    def __repr__(self):
        return f'BTreeNode(keys={self.keys}, leaf={self.leaf})'

    # 3. Przeczytaj stronę wskazaną przez s do pamięc (ALGORYTM WYSZUKIWANIA)

    # 4. Poszukaj x na tej stronie (ALGORYTM WYSZUKIWANIA)

    # Znajduje i zwraca indeks, w którym klucz k powinien się znaleźć w liście kluczy węzła.

    # szukam indeksu, pod którym klucz k powinien być
    # lub gdzie mógłby się znajdować w liście kluczy
    def find_key(self, k):

        idx = 0
        # przechodze przez listę kluczy w węźle, aż znajde klucz większy lub równy k
        while idx < len(self.keys) and self.keys[idx] < k:
            idx += 1
        # zwracam indeks, który wskazuje, gdzie k powinien być
        # lub gdzie można kontynuować wyszukiwanie w dzieciach węzła
        return idx


    # metoda insert_non_full() - wstawia klucz do węzła, który nie jest pełny,
    # obsługując zarówno węzły liściowe, jak i wewnętrzne
    # jeśli węzeł dziecka jest pełny, wywołuje split_child w celu podziału
    def insert_non_full(self, k):

        # inicjalizacja wskaźnika
        # i to wskaźnik na ostatni klucz w węźle
        # rozpoczynam od ostatniego klucza i będziemy przesuwać w lewo,
        # aby znaleźć odpowiednią pozycję dla nowego klucza
        i = len(self.keys) - 1

        # jeśli węzeł jest liściem
        if self.leaf:
            # dodaje miejsce na nowy klucz
            self.keys.append(None)
            # przeszukuje klucze od końca, aby znaleźć
            # odpowiednią pozycję, gdzie k powinien być wstawiony
            while i >= 0 and self.keys[i] > k:
                # klucze większe od k są przesuwane w prawo, aby zrobić miejsce
                self.keys[i + 1] = self.keys[i]
                i -= 1
            # po znalezieniu właściwej pozycji, klucz k jest wstawiany w odpowiednie miejsce
            self.keys[i + 1] = k

        # jeśli węzeł nie jest liściem
        else:
            # znajduje odpowiednią pozycję w liście kluczy, aby określić, do którego dziecka należy wstawić klucz k
            while i >= 0 and self.keys[i] > k:
                # przesuwam wskaźnik 'i' w lewo, aż znajde klucz, który nie jest większy niż k
                i -= 1

            # Po zakończeniu pętli 'i' wskazuje na indeks klucza, który jest mniejszy lub równy k
            # musze dodać 1 do i, aby wskazać odpowiednie dziecko.
            # i += 1 zapewnia, że wskazujemy na dziecko, które znajduje się bezpośrednio po kluczu,
            # który jest mniejszy lub równy k
            i += 1

            # Sprawdzam, czy wybrane dziecko (self.children[i]) jest pełne, tzn. ma już maksymalną liczbę kluczy,
            # która wynosi 2 * t - 1. (CHYBA JEDNAK 2 * t)
            if len(self.children[i].keys) == 2 * self.t - 1:
                # jeśli tak,to musze podzielić to dziecko, aby zrobić miejsce na nowe klucze
                # wywołuje 'self.split_child(i)' aby podzielić pełne dziecko na dwa węzły
                self.split_child(i)

                # Po podziale, klucz przeniesiony do węzła rodzica (self.keys[i]) może być mniejszy lub większy od klucza k.

                # Jeśli klucz w węźle rodzica jest mniejszy od k, oznacza to, że nowy klucz k powinien być wstawiony do
                # drugiego z nowo utworzonych dzieci, więc i += 1 zapewnia, że wskazujemy na właściwe dziecko
                if self.keys[i] < k:
                    i += 1


            # Rekurencyjne wstawienie

            # wywołuje insert_non_full na odpowiednim dziecku (self.children[i]), aby wstawić klucz k do tego dziecka.
            # Jeśli to dziecko również nie jest liściem, procedura rekurencyjnie znajdzie odpowiednie miejsce i ewentualnie podzieli swoje dzieci
            self.children[i].insert_non_full(k)

    # dziele pełne dziecko węzła na dwa węzły i aktualizuje klucze oraz dzieci węzła rodzica
    def split_child(self, i):
        # t to minimalny stopień B-drzewa
        t = self.t
        # y to węzeł dziecka, który jest pełny i zostanie podzielony
        y = self.children[i]
        # z to nowy węzeł, który będzie przechowywał część kluczy z y
        z = BTreeNode(t, y.leaf)
        # dodaje nowy węzeł z jako dziecko węzła rodzica
        self.children.insert(i + 1, z)
        # przenosi klucz, który oddziela dwa węzły do węzła rodzica
        self.keys.insert(i, y.keys[t - 1])
        # przypisuje klucze z y do nowego węzła z
        z.keys = y.keys[t:(2 * t) - 1]
        # aktualizuje klucze w oryginalnym węźle y, usuwając przeniesione klucze
        y.keys = y.keys[0:t - 1]

        # Sprawdzenie, czy y nie jest liściem
        # jeśli y jest liściem (y.leaf jest True), to nie ma dzieci, więc nie musze przenosić żadnych dzieci do nowego węzła 'z'
        if not y.leaf:

            # przenoszenie dzieci z y do z
            # po podziale, y powinien mieć teraz mniej dzieci, a nowy węzeł 'z' powinien mieć pozostałe dzieci, które były w y po podziale
            # y.children[t:(2 * t)] wybiera dzieci od indeksu t do końca listy dzieci y.
            # jest to zakres, który obejmuje dzieci od miejsca podziału do końca
            z.children = y.children[t:(2 * t)]

            # aktualizacja listy dzieci w y, aby zawierała tylko dzieci, które pozostały w 'y' po podziale
            # po przeniesieniu części dzieci do 'z', w 'y' powinny pozostać tylko dzieci od początku do miejsca podziału
            # y.children[0:t - 1] wybiera dzieci od początku do indeksu t - 1.
            # jest to zakres dzieci, które pozostają w y po podziale
            y.children = y.children[0:t - 1]

class BTree:
    def __init__(self, t, path, buffer_size=4):
        self.root = BTreeNode(t, leaf=True)
        self.t = t
        self.read_operations = 0
        self.write_operations = 0
        self.path = path
        self.read_buffer = ReadBuffer(path, buffer_size)
        self.write_buffer = WriteBuffer(path, buffer_size)
        self.buffer_size = buffer_size
        # tu chyba bez tego -1
        self.max_keys = 2 * t - 1  # maksymalna liczba kluczy w węźle

    # 1. Wstawianie Rekordu

    # wstawiam rekord do B drzewa , sprawdzając czy korzeń jest pełny i reorganizując drzewo, jeśli liczba kluczy przekracza limit
    def insert(self, record):
        # szukam rekordu z określonym kluczem w drzewie, czy rekord już istnieje
        if self.search(record.key) is not None:
            # jeśli tak, to wypisuje komunikat i kończe działanie metody
            print(f'Already exists: {record.key}')
            return

        # pobieram korzeń drzewa
        root = self.root
        # sprawdzam, czy korzeń drzewa jest pełny (czy zawiera maksymalną liczbę kluczy)
        if len(root.keys) == self.max_keys:
            # tworze nowy węzeł jako tymczasowy korzeń
            temp = BTreeNode(self.t)
            # nowy węzeł tymczasowy staje się korzeniem drzewa
            self.root = temp
            # przenosze stary korzeń jako pierwsze dziecko nowego korzenia
            temp.children.insert(0, root)
            # dziele stary korzeń i wstawiam klucz do odpowiedniego węzła
            self._split_child(temp, 0)
            # wstawiam rekord do nowego korzenia (który może być teraz pełny)
            self._insert_non_full(temp, record)
        else:
            # wstawiam rekord do węzła, który nie jest pełny
            self._insert_non_full(root, record)
        # zwiększa licznik operacji zapisu
        self.write_operations += 1
        # zapisuje rekord do bufora zapisu
        self.write_buffer.write(record)

        # 6. Automatyczna Reorganizacja

        # sprawdzam, czy liczba kluczy w całym drzewie przekracza dwukrotność maksymalnej liczby kluczy w węźle
        if len(self._get_all_keys(self.root)) > 2 * self.max_keys:
            # jeśli tak, to reorganizuje drzewo
            self.reorganize()

    # wstawiam klucz rekordu do węzła, który nie jest pełny, przy pomocy metody insert_non_full z BTreeNode
    def _insert_non_full(self, node, record):
        # wstawiam klucz rekordu do węzła, który nie jest pełny (wywołuje metodę insert_non_full w BTreeNode)
        node.insert_non_full(record.key)
        # Zwiększa licznik operacji zapisu
        self.write_operations += 1

    # dziele pełne dziecko węzła i aktualizuje struktury drzewa, zwiększając licznik operacji zapisu
    def _split_child(self, node, i):
        # dziele pełne dziecko węzła w indeksie i (wywołuje metodę split_child w BTreeNode)
        node.split_child(i)
        # zwiększa licznik operacji zapisu
        self.write_operations += 1

    # 2. Odczyt Rekordu

    # odczytuje rekord o określonym kluczu z pliku przy pomocy bufora odczytu
    def read_record(self, key):
        # zwiększam licznik operacji odczytu
        self.read_operations += 1
        # ustawiam wskaźnik pliku na początek
        self.read_buffer.file.seek(0)

        # pętla do odczytu rekordów z bufora.
        while True:
            # czytam następny rekord z bufora
            record = self.read_buffer.read_next()
            # sprawdzam, czy rekordy się skończyły
            if record is None:
                break
            # sprawdzam, czy klucz rekordu jest równy poszukiwanemu kluczowi
            if record.key == key:
                # zwracam rekord, jeśli klucz jest zgodny
                return record
        # zwracam None, jeśli rekord z poszukiwanym kluczem nie został znaleziony
        return None

    # 3. Przeglądanie Całej Zawartości Pliku i Indeksu

    # rozpoczynam przeglądanie B drzewa, wywołując metodę _traverse do drukowania wszystkich kluczy w porządku in-order.
    def traverse(self):
        # rozpoczynam przeglądanie od korzenia
        self._traverse(self.root)
        # dodaje pusty wiersz po zakończeniu przeglądania
        print()


    # rekursywnie przeglądam i drukuje klucze w B drzewie, wywołując metodę dla dzieci węzła
    def _traverse(self, node):
        # przechodze przez klucze węzła i wywołuje rekursywnie dla dzieci
        for i in range(len(node.keys)):
            # sprawdzam, czy węzeł nie jest liściem
            if not node.leaf:
                # wywołuje rekursywnie _traverse dla każdego dziecka węzła
                self._traverse(node.children[i])
            # drukuje klucz węzła
            print(node.keys[i], end=' ')
        # jeśli po iteracji przez klucze, węzeł nie jest liściem
        if not node.leaf:
            # wywołuje rekursywnie _traverse dla ostatniego dziecka węzła
            self._traverse(node.children[len(node.keys)])

    # ALGORYTM WYSZUKIWANIA

    # wyszukuje klucz w B drzewie, zwiększając licznik operacji odczytu
    def search(self, k):
        # zwiększam licznik operacji odczytu
        self.read_operations += 1

        # wywołuje prywatną metodę _search do wyszukiwania klucza w drzewie
        # 1. Niech s oznacza adres strony korzenia (ALGORYTM Z PDF)
        # s to self.root
        return self._search(self.root, k)

    # funkcja _search jest wywoływana rekurencyjnie, co odpowiada krokom algorytmu przechodzenia
    # do odpowiedniego dziecka i kontynuowania wyszukiwania
    # rekursywnie wyszukuje klucz w drzewie B, zwracając wynik, jeśli klucz zostanie znaleziony
    def _search(self, node, k):

        # 3. Przeczytaj stronę wskazaną przez s do pamięci (ALGORYTM Z PDF)
        # znajduje indeks klucza, który może być równy k
        i = node.find_key(k)

        # 5. Jeżeli znaleziono xi = x, to RETURN (xi, ai)
        # sprawdzam, czy klucz znajduje się w bieżącym węźle
        if i < len(node.keys) and node.keys[i] == k:
            # zwraca klucz i dziecko (jeśli węzeł nie jest liściem)
            return (node.keys[i], node.children[i] if not node.leaf else None)

        # jeśli węzeł jest liściem
        if node.leaf:
            # klucz nie został znaleziony
            return None
        # kontynuuje wyszukiwanie w odpowiednim dziecku
        return self._search(node.children[i], k)

    # 4. Reorganizacja Pliku

    # reorganizuje B drzewo, sortując wszystkie klucze i ładując je do nowego drzewa
    def reorganize(self):
        # pobieram wszystkie klucze z drzewa
        all_keys = self._get_all_keys(self.root)
        # sortuje klucze
        all_keys.sort()
        # tworze nowy korzeń jako liść
        self.root = BTreeNode(self.t, leaf=True)
        # ładowanie posortowanych kluczy do nowego drzewa
        self._bulk_load(all_keys)

    # zbieram i zwracam wszystkie klucze z B drzewa, w uporządkowanej liście
    def _get_all_keys(self, node):
        # inicjalizuje pustą listę kluczy
        keys = []
        # sprawdzam, czy węzeł jest liściem
        if node.leaf:
            # dodaje klucze liścia do listy
            keys.extend(node.keys)
        # jeśli węzeł nie jest liściem, przetwarza dzieci
        else:
            # iteruje przez klucze węzła
            for i in range(len(node.keys)):
                # dodaje klucze z dziecka
                # extend - bo rozszerzam o kilka (a nie tylko o ten jeden(gorny))
                keys.extend(self._get_all_keys(node.children[i]))
                # dodaje klucz bieżącego węzła
                keys.append(node.keys[i])
            # dodaje klucze z ostatniego dziecka
            keys.extend(self._get_all_keys(node.children[len(node.keys)]))
        # zwracam listę wszystkich kluczy
        return keys

    # wstawiam posortowane klucze do B drzewa jako nowe rekordy
    def _bulk_load(self, keys):
        # iteruje przez posortowane klucze
        for key in keys:
            # wstawiam każdy klucz jako nowy rekord z pustą wartością
            self.insert(Record(key, ''))

    # 5. Wyświetlanie Zawartości Pliku i Indeksu

    # drukuje strukturę B drzewa, pokazując klucze w każdym poziomie drzewa
    def print_structure(self):
        # wywołuje prywatną metodę do drukowania struktury drzewa, zaczynając od korzenia na poziomie 0
        self._print_structure(self.root, 0)

    # rekursywnie drukuje klucze węzła i jego poziom w B drzewie, wywołując metodę dla dzieci
    def _print_structure(self, node, level):
        # drukuje klucze węzła i jego poziom
        print("Level", level, ":", node.keys)
        # sprawdza, czy węzeł nie jest liściem
        if not node.leaf:
            # iteruje przez dzieci węzła
            for child in node.children:
                # rekursywnie wywołuje _print_structure dla każdego dziecka, zwiększając poziom o 1
                self._print_structure(child, level + 1)

    def close_buffers(self):
        # zamykam bufor odczytu
        self.read_buffer.close()
        # zamykam bufor zapisu
        self.write_buffer.close()

# 7. Interaktywny i Plikowy Tryb Działania

def process_commands(btree, commands):
    for command in commands:
        # usuwam białe znaki z początku i końca komendy, a następnie dziele komendę na części na podstawie spacji
        parts = command.strip().split()
        # sprawdzam, czy lista części (parts) jest pusta (czyli komenda była pusta lub zawierała tylko białe znaki)
        if not parts:
            continue
        # sprawdzam, czy pierwsza część komendy to insert
        if parts[0] == 'insert':
            # pobieram klucz jako liczbę całkowitą z drugiej części komendy
            key = int(parts[1])
            # pobieram dane z trzeciej części komendy
            data = parts[2]
            # tworze nowy rekord z kluczem i danymi.
            record = Record(key, data)
            # wstawiam rekord do B drzewa
            btree.insert(record)
            # informuje o dodaniu klucza do drzewa
            print(f"Inserted {key}.")
            # wyświetlam strukturę drzewa po dodaniu rekordu
            btree.print_structure()
        # sprawdzam, czy pierwsza część komendy to find
        elif parts[0] == 'find':
            # pobiera klucz jako liczbę całkowitą
            key = int(parts[1])
            # wyszukuje klucz w B drzewie
            found = btree.search(key)
            # sprawdzam, czy klucz został znaleziony
            if found:
                # jeśli klucz został znaleziony, pobieram rekord z drzewa
                record = btree.read_record(key)
                # wyświetlam znaleziony rekord
                print(f"Found: {record}")
            # informuje, że klucz nie został znaleziony
            else:
                print(f"Key {key} not found.")

        # Metoda traverse jest używana do wyświetlania wszystkich kluczy w B-drzewie w uporządkowany sposób (in-order traversal).
        # Dzięki temu można  zobaczyć wszystkie klucze w drzewie w kolejności rosnącej,
        # co jest pomocne do weryfikacji struktury drzewa i poprawności jego organizacji

        # sprawdzam, czy pierwsza część komendy to traverse
        elif parts[0] == 'traverse':
            # wywołuje metodę traverse na drzewie B, aby wyświetlić wszystkie klucze w kolejności rosnącej
            btree.traverse()
        # sprawdzam, czy pierwsza część komendy to stats
        elif parts[0] == 'stats':
            # wyświetla liczbę operacji odczytu i zapisu wykonanych na B drzewie
            print(f"Read operations: {btree.read_operations}, Write operations: {btree.write_operations}")
        # sprawdzam, czy pierwsza część komendy to reorganize
        elif parts[0] == 'reorganize':
            # wywołuje metodę reorganize, aby zorganizować B drzewo
            btree.reorganize()
            print("Reorganized the B-tree.")
            # wyświetlam nową strukturę drzewa po reorganizacji
            btree.print_structure()
        # sprawdzam, czy pierwsza część komendy to print
        elif parts[0] == 'print':
            # wywołuje metodę print_structure, aby wyświetlić aktualną strukturę drzewa
            btree.print_structure()
        else:
            print(f"Unknown command: {command}")

def read_commands_from_file(filename):
    # otwieram plik o nazwie filename w trybie odczytu
    with open(filename, 'r') as file:
        # zwracam wszystkie linie z pliku jako listę
        return file.readlines()

def main():
    path = 'test'
    # definiuje rozmiar bufora
    buffer_size = 4
    # tworze obiekt drzewa B z minimalnym stopniem t = 2
    btree = BTree(2, path, buffer_size)
    # pobieram tryb działania od użytkownika
    mode = input("Enter 'file' to read commands from a file or 'interactive' for interactive mode: ")
    # sprawdzam, czy tryb to file
    if mode == 'file':
        # pobieram nazwę pliku z komendami
        filename = input("Enter the filename: ")
        # odczytuje komendy z pliku
        commands = read_commands_from_file(filename)
        # przetwarzam odczytane komendy
        process_commands(btree, commands)

    elif mode == 'interactive':
        while True:
            command = input("Enter command: ")
            if command == 'exit':
                break
            # przetwarzam pojedynczą komendę
            process_commands(btree, [command])
    # zamykam bufory odczytu i zapisu po zakończeniu działania.
    btree.close_buffers()

if __name__ == "__main__":
    main()

Enter 'file' to read commands from a file or 'interactive' for interactive mode: interactive
Enter command: insert 10 42
Inserted 10.
Level 0 : [10]
Enter command: insert 20 32
Inserted 20.
Level 0 : [10, 20]
Enter command: insert 30 53
Inserted 30.
Level 0 : [10, 20, 30]
Enter command: insert 50 78
Inserted 50.
Level 0 : [20]
Level 1 : [10]
Level 1 : [30, 50]
Enter command: insert 10 54
Already exists: 10
Inserted 10.
Level 0 : [20]
Level 1 : [10]
Level 1 : [30, 50]
Enter command: exit
